## Heart Disease Prediction


Author: Lana Cartailler
Notes:
1. Install [Graphviz](https://graphviz.gitlab.io/download/) - for the neural network plot
2. To consider, the biggest risk factors for heart disease are BMI, diabetes, smoking, low physical activity, and increased age.

In [1]:
# Configuration
data_file_path = 'heart_2020_cleaned.csv'
use_class_weights = True
use_feature_weights = False

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.initializers import Constant
from keras.utils import to_categorical # still need?
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import matplotlib.pyplot as plt

## Load data and create features and target dataframes

In [3]:
data = pd.read_csv(data_file_path)
data.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [4]:
# Split data into dataframes for features (X) and target (y)
# Remember, y = f(X)
y = data['HeartDisease'] # factor being predicted
X = data.drop('HeartDisease', axis=1) # .drop keeps all other columns besides target column

## Data 

1. Explore data
2. Determine which columns need to be one-hot encoded
3. Determine which columns need to be scaled
4. Split data into training and testing sets

In [5]:
# Determine which features need to be one-hot encoded, binary encoded, or scaled
# for non-numeric columns, get # of unique values to determine which ones need to be one-hot encoded
non_numeric_cols = X.select_dtypes(exclude='number').columns # .columns retrieves column headers that aren't numbers

for col in non_numeric_cols:
    r = X[col].nunique() # number of unique values in column
    if(r > 2):
        onehot = 'one-hot encode this one'
    else:
        onehot = 'binary encode this one'
        
    print(f'{col:<15}: {r} unique values ({onehot})')
    
numeric_cols = X.select_dtypes(include='number').columns # retrieves column headers that are numbers
for col in numeric_cols:
    print(f'{col:<15}: is numeric (scale it)')

Smoking        : 2 unique values (binary encode this one)
AlcoholDrinking: 2 unique values (binary encode this one)
Stroke         : 2 unique values (binary encode this one)
DiffWalking    : 2 unique values (binary encode this one)
Sex            : 2 unique values (binary encode this one)
AgeCategory    : 13 unique values (one-hot encode this one)
Race           : 6 unique values (one-hot encode this one)
Diabetic       : 4 unique values (one-hot encode this one)
PhysicalActivity: 2 unique values (binary encode this one)
GenHealth      : 5 unique values (one-hot encode this one)
Asthma         : 2 unique values (binary encode this one)
KidneyDisease  : 2 unique values (binary encode this one)
SkinCancer     : 2 unique values (binary encode this one)
BMI            : is numeric (scale it)
PhysicalHealth : is numeric (scale it)
MentalHealth   : is numeric (scale it)
SleepTime      : is numeric (scale it)


In [6]:
# One-hot encode the data using pandas get_dummies
X = pd.get_dummies(X, columns=['AgeCategory', 'Race']) # creates new column for each unique value for each column & removes OG column
X = pd.get_dummies(X, columns=['Diabetic', 'GenHealth'])

# Binary-encode
X['Smoking'] = X['Smoking'].replace({'Yes': 1, 'No': 0})
X['AlcoholDrinking'] = X['AlcoholDrinking'].replace({'Yes': 1, 'No': 0})
X['Stroke'] = X['Stroke'].replace({'Yes': 1, 'No': 0})
X['DiffWalking'] = X['DiffWalking'].replace({'Yes': 1, 'No': 0})
X['Sex'] = X['Sex'].replace({'Male': 1, 'Female': 0})
X['PhysicalActivity'] = X['PhysicalActivity'].replace({'Yes': 1, 'No': 0})
X['Asthma'] = X['Asthma'].replace({'Yes': 1, 'No': 0})
X['KidneyDisease'] = X['KidneyDisease'].replace({'Yes': 1, 'No': 0})
X['SkinCancer'] = X['SkinCancer'].replace({'Yes': 1, 'No': 0})

In [7]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=666) # creates 2 dfs & 2 lists. 10% of 319k rows randomly chosen for testing & 90% reserved for training. random_state is seed used to generate same randomly chosen rows

In [8]:
# Scale continuous variables
# eliminates dominating features that might dominate the algo but maintains scale within each feature (normalizes)
scaler = StandardScaler()
X_train[['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']] = scaler.fit_transform(X_train[['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']]) 
X_test[['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']] = scaler.transform(X_test[['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']])

In [9]:
# Convert input data to float data type (otherwise get later error)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [10]:
# Convert target variable to numerical values
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Model preparation and training

In [11]:
if(use_class_weights == True):
    # Calculate class weights
    # Class weights are used to address imbalanced data. 
    # The class with the fewest samples will have the highest weight, 
    # and the class with the most samples will have the lowest weight.
    # The weights are calculated as: n_samples / (n_classes * np.bincount(y))
    # where n_samples is the total number of samples in the dataset, 
    # n_classes is the total number of classes in the dataset, and 
    # np.bincount(y) is the number of samples for each class.
    n_samples = len(y_train) # only want to balance target
    n_classes = 2
    y_train = y_train.astype(np.int32)
    bincount = np.bincount(y_train) # get rid of bracket [:, 1]?
    weights = n_samples / (n_classes * bincount)
    weights = {0: weights[0], 1: weights[1]}
else:
    weights = None
    
weights
# 1s are 10x more weighted than 0s bc we know that there are 10x more no's than yeses

{0: 0.5467153201480119, 1: 5.8515634530150855}

In [12]:
neurons_in_first_hidden_layer = round((X_train.shape[1] + 1) / 2) # average of input and output layers

In [13]:
# Define default weight for features not in the dictionary
default_weight = 1
weights_dict = {}

if(use_feature_weights == True):    
    # Define custom weights for specific features    
    weights_dict = {'BMI': 6,
                    'Smoking': 6,
                    'PhysicalHealth': 3,
                    'Diabetic_Yes': 3,
                    'Diabetic_No': 3,
                    'PhysicalActivity': 4,
                    'GenHealth_Excellent': 4,
                    'GenHealth_Very good': 2,
                    'GenHealth_Good': 2,
                    'GenHealth_Fair': 2,
                    'GenHealth_Poor': 4
                    }
    
# Define list of weights for all features
feature_weights = [weights_dict.get(col, default_weight) for col in X_test.columns]
weights_2d = np.tile(feature_weights, neurons_in_first_hidden_layer) 
weights_2d = np.reshape(weights_2d, (41, neurons_in_first_hidden_layer))
weights_2d

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [14]:
# Define model architecture
model = Sequential() # allows model to build layer by layer
model.add(Dense(neurons_in_first_hidden_layer, 
                input_dim=X_train.shape[1], 
                activation='relu', 
                kernel_initializer=Constant(weights_2d))) # input dimensions are 41 - number of columns in dataframe used for input (boxes).

model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # want number of neurons to match number of final desired responses. Activation function returns probability of yes & no that adds to 1.

In [15]:
# Compile model
opt = Adam(learning_rate=0.001) # default is 0.001, .001 to 0.0001 is a good start

model.compile(loss='binary_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.BinaryCrossentropy()]) # puts together model. Loss function - measures how well model can predict the true labels (known yeses/nos of original data) of training data; want model to minimize loss function, which will lead to better predictions. Metrics track performance.

In [16]:
# Train model
# As training progresses, the loss should drop and the accuracy should increase.
history = model.fit(X_train, y_train, epochs=25, batch_size=50, validation_data=(X_test, y_test), class_weight=weights) # epoch - single interation where each sample in dataset is used once to update model parameters internally. Epochs are used multiple times so model can learn & improve accuracy. Batch size - how many samples/number of rows get pushed thru network at any one time. Validation data - internal checks. Class weights - weights calculated earlier for yeses/nos for heart disease

Epoch 1/25
5757/5757 [==============================] - 61s 10ms/step - loss: 0.5174 - accuracy: 0.7309 - precision: 0.2088 - recall: 0.7707 - binary_crossentropy: 0.5312 - val_loss: 0.6660 - val_accuracy: 0.6478 - val_precision: 0.1829 - val_recall: 0.8799 - val_binary_crossentropy: 0.6660
Epoch 2/25
5757/5757 [==============================] - 54s 9ms/step - loss: 0.4997 - accuracy: 0.7265 - precision: 0.2107 - recall: 0.8018 - binary_crossentropy: 0.5102 - val_loss: 0.5972 - val_accuracy: 0.6513 - val_precision: 0.1840 - val_recall: 0.8766 - val_binary_crossentropy: 0.5972
Epoch 3/25
1475/5757 [======>.......................] - ETA: 32s - loss: 0.4956 - accuracy: 0.7236 - precision: 0.2109 - recall: 0.8123 - binary_crossentropy: 0.5078

KeyboardInterrupt: 

In [ ]:
# Plot training history

plt.figure(figsize = (16,4))
# Accuracy - 
# If trend for both datasets is still rising for last few epochs, it can be trained more.
# If both curves are comparable, then it isn't overfit, otherwise over-learned the training dataset
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# Loss
# Comparable performance on both train and validation datasets is good. 
# If parallel plots start to depart, it might be a sign to stop training at an earlier epoch.
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Evaluate model based on test data
loss, accuracy, precision_12, recall_12, binary_crossentropy = model.evaluate(X_test, y_test)

print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('Test precision:', precision_12)
print('Test recall:', recall_12)
print('Test binary_crossentropy:', binary_crossentropy)

## Use model to make predictions

### Scenario A - Assume 0.5 is threshold between NO and YES predictions

In [ ]:
# Make predictions
#
# The predict method of the Keras model takes the test data as input and returns the predicted y values for each sample in the test data (X_test). The output of the predict method is an array of predicted values for each sample in the test data. The shape of the output array is (n_samples, n_classes), where n_samples is the number of samples in the test data and n_classes is the number of classes in the target variable.

predictions = model.predict(X_test)

In [ ]:
# Assuming that the target variable is binary (i.e., has two classes), the 
# first column of the output array represents the predicted probability of the
# sample belonging to the first class, and the second column represents the 
# predicted probability of the sample belonging to the second class.
# Get predicted class labels

y_pred = np.round(predictions)
y_pred = np.round(y_pred).astype('int64')

predict_count_no = np.bincount(y_pred[:,0])[0]
predict_count_yes = np.bincount(y_pred[:,0])[1]

print('Number of predictions for NO heart disease:', predict_count_no)
print('Number of predictions for YES heart disease:', predict_count_yes)

In [ ]:
# Expected output (truth) from original X_test data selection, mapped back to
# original data with HeartDisease column
test_index_list = X_test.index # retrieves row indices that are inside X test data stored as list
data_matching_test = data.loc[test_index_list] # retrieves rows based on index location. Gets OG data from .csv file & pulls rows that correspond to test data. Stores in dataframe.
data_matching_test['HeartDisease'].value_counts() # finds all unique values (yeses and nos) in HeartDisease column & counts how many of each there are

In [ ]:
# Create dataframe with real and predicted values side by side
df = pd.DataFrame({'Real': data_matching_test['HeartDisease'], 'Predicted': y_pred[:, 0]})

HeartDisease_count_predicted = df[(df['Real'] == 'Yes') & (df['Predicted'] == 1)].shape[0]
HeartDisease_count_truth = data_matching_test['HeartDisease'].value_counts().iloc[1]
percentage = round((HeartDisease_count_predicted / HeartDisease_count_truth)*100, ndigits=1)
print("In terms of accuracy, this model predicted " + str(HeartDisease_count_predicted) + " out of " + str(HeartDisease_count_truth) + " subjects (" + str(percentage) + "%) that were diagnosed with heart disease.") 
print("However, in terms of precision, the model suffered from a high rate of false positives. It predicted " + str(predict_count_yes) + " subjects did have heart disease, but only " + str(HeartDisease_count_truth) + " of those subjects actually had heart disease.")

### Scenario B - Assume threshold between NO and YES predictions is NOT 0.5

In [ ]:
probabilities = model.predict(X_test)

In [ ]:
threshold = 0.55
y_pred = (probabilities > threshold).astype('int')

In [ ]:
predict_count_no = np.bincount(y_pred[:,0])[0]
predict_count_yes = np.bincount(y_pred[:,0])[1]

print('Number of predictions for NO heart disease:', predict_count_no)
print('Number of predictions for YES heart disease:', predict_count_yes)

In [ ]:
# Create dataframe with real and predicted values side by side
df = pd.DataFrame({'Real': data_matching_test['HeartDisease'], 'Predicted': y_pred[:, 0]})

HeartDisease_count_predicted = df[(df['Real'] == 'Yes') & (df['Predicted'] == 1)].shape[0]
HeartDisease_count_truth = data_matching_test['HeartDisease'].value_counts().iloc[1]
percentage = round((HeartDisease_count_predicted / HeartDisease_count_truth)*100, ndigits=1)
print("In terms of accuracy, this model predicted " + str(HeartDisease_count_predicted) + " out of " + str(HeartDisease_count_truth) + " subjects (" + str(percentage) + "%) that were diagnosed with heart disease.") 
print("However, in terms of precision, the model suffered from a high rate of false positives. It predicted " + str(predict_count_yes) + " subjects did have heart disease, but only " + str(HeartDisease_count_truth) + " of those subjects actually had heart disease.")

## Conclusions

A neural network was a moderately successful model for predicting heart disease from 17 clinical attributes (features). The model was able to accurately predict about 80% of the cases, but it exhibits low precision in the form of false positives, identifying roughly 3 times too many subjects as having heart disease who did not.

In [ ]:
y = data[['BMI', 'HeartDisease']].reset_index(drop=True)
y.head()

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='BMI and Heart Disease', style={'textAlign':'center'}),
    dcc.Graph(id='graph-content')
])

# @callback(
#    Output('graph-content', 'figure'),
#    Input('dropdown-selection', 'value')
# )
def update_graph(value):
    return px.bar(y, x='HeartDisease', y='BMI')

if __name__ == '__main__':
    app.run(debug=True)